# Lab 6-2: Fancy Softmax Classification

Author: Seungjae Lee (이승재)

## Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# For reproducibility
torch.manual_seed(1)

## Cross-entropy Loss with `torch.nn.functional`

PyTorch has `F.log_softmax()` function.

In [ ]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
y = torch.randint(5, (3,)).long()
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [ ]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood.

In [ ]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y.long())

tensor(1.4689, grad_fn=<NllLossBackward0>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [ ]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

## Data

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
xy = np.loadtxt('/content/drive/MyDrive/data-04-zoo.csv', delimiter=',', dtype=np.float32)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
x_train = torch.FloatTensor(xy[:, 0:-1])
y_train = torch.LongTensor(xy[:, [-1]]).squeeze()

In [ ]:
print(x_train.shape) # x_train shape
print(len(x_train))  # x_train 길이
print(x_train[:5])   # 첫 다섯 개

torch.Size([101, 16])
101
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [ ]:
print(y_train.shape) # y_train shape
print(len(y_train))  # y_train 길이
print(y_train[:5])   # 첫 다섯 개

torch.Size([101])
101
tensor([0, 0, 3, 0, 0])


In [ ]:
nb_classes = 7 # csv 파일을 보시면 0~6까지 총 7개의 class인 것을 확인할 수 있습니다!
y_one_hot = torch.zeros((len(y_train), nb_classes))
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

## Training with `F.cross_entropy`

In [ ]:
# 모델 초기화
W = torch.zeros([16,101],requires_grad=True) #가중치 W를 0으로 초기화하고 학습을 통해 값이 변경될 수 있도록 하세요.
b = torch.zeros(1,requires_grad=True) #편향 b를 0으로 초기화하고 학습을 통해 값이 변경될 수 있도록 하세요.
# optimizer 설정
optimizer =optim.SGD([W,b],lr=0.1) #SGD optimizer를 사용하고 learning rate는 0.1로 적용하세요.

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z,y_train) #PyTorch에서 제공하는 F.cross_entropy함수를 사용해 cost를 구현해보세요.

    # cost로 H(x) 개선
    optimizer.zero_grad()  # gradient를 0으로 초기화
    cost.backward()  # 비용 함수를 미분하여 gradient 계산
    optimizer.step()  # W와 b를 업데이트

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 4.615120
Epoch  100/1000 Cost: 0.620671
Epoch  200/1000 Cost: 0.413353
Epoch  300/1000 Cost: 0.323833
Epoch  400/1000 Cost: 0.269756
Epoch  500/1000 Cost: 0.232296
Epoch  600/1000 Cost: 0.204252
Epoch  700/1000 Cost: 0.182234
Epoch  800/1000 Cost: 0.164370
Epoch  900/1000 Cost: 0.149517
Epoch 1000/1000 Cost: 0.136935


## High-level Implementation with `nn.Module`

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16,101 ) #x_train의 shape과 y_train의 class의 갯수를 고려해보세요.
    def forward(self, x):
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel() #SoftmaxClassifierModel()로 모델을 초기화하세요.

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr=0.1) #SGD optimizer를 사용하고 learning rate는 0.1로 적용하세요.

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train) #모델을 생성해서 예측값을 구해보세요.

    # cost 계산
    cost = F.cross_entropy(prediction,y_train) #PyTorch에서 제공하는 F.cross_entropy함수를 사용해 cost를 구현해보세요.

    # cost로 H(x) 개선
    optimizer.zero_grad()  # gradient를 0으로 초기화
    cost.backward()  # 비용 함수를 미분하여 gradient 계산
    optimizer.step()  # W와 b를 업데이트
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 4.608705
Epoch  100/1000 Cost: 0.584688
Epoch  200/1000 Cost: 0.386777
Epoch  300/1000 Cost: 0.295570
Epoch  400/1000 Cost: 0.239615
Epoch  500/1000 Cost: 0.201335
Epoch  600/1000 Cost: 0.173435
Epoch  700/1000 Cost: 0.152177
Epoch  800/1000 Cost: 0.135434
Epoch  900/1000 Cost: 0.121913
Epoch 1000/1000 Cost: 0.110777


<div class="alert alert-warning">
    Should I display how many it got correct in the training set?
</div>

Q1. 각 Regression에 맞는 Hypothesis와 cost function을 적어주세요.

Linear Regression : Wx+b , sigma(H(x)-y)^2

Logistic Regression : 1/(1+e^(-z)) ,-ylog(h(x))-(1-y)log(1-h(x))

Multi Logistic regression : 1/(1+e^(-WTX)) , -ylog(h(x))-(1-y)log(1-h(x))

Q2. Lec06-2의 Logistic cost VS Cross entropy에서 두 함수가 같은 이유를 서술해주세요.

=> logistic regression은 비선형인 sigmoid 함수를 사용하게 된다. 비선형을 출력할경우 MSE를 사용할 수 없다. 따라서 cross entropy를 사용해야한다.